# Setup

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://pepy.tech/project/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [ ]:
# %pip install ultralytics
import ultralytics

ultralytics.checks()

In [ ]:
# @title Select YOLOv8 🚀 logger {run: 'auto'}
logger = 'Comet'  # @param ['Comet', 'TensorBoard']

if logger == 'Comet':
    %pip install -q comet_ml
    import comet_ml

    comet_ml.init()
elif logger == 'TensorBoard':
    %load_ext tensorboard
    %tensorboard --logdir .

# 1. Train


In [ ]:
# Train YOLOv8n on rosatom dataset
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8m.yaml')  # build a new model from YAML
model = YOLO('yolov8m.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8m.yaml').load(
    'yolov8m.pt'
)  # build from YAML and transfer weights

# Train the model
results = model.train(data='rosatom.yaml', epochs=200, imgsz=640)

# 2. Prediction

In [ ]:
import os
import csv
from ultralytics import YOLO


DIR_IMAGES = './data/dataset/rosatom/images/val/'


# Run batched inference on a list of images
# list_images = ["im1.jpg", "im2.jpg"]
files = [os.path.join(DIR_IMAGES, f) for f in os.listdir(DIR_IMAGES)]
print(f'{len(files)} files for submission')
# Load a model
model = YOLO('yolov8_n_200epoch.pt')

results = model(files)  # return a list of Results objects

In [ ]:
# Define CSV file path
import random

import csv

csv_file = 'submission.csv'

# Write results to CSV
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(
        ['filename', 'class_id', 'rel_x', 'rel_y', 'width', 'height']
    )
    for result in results:
        file_name = os.path.basename(result.path)
        for bbox in result.boxes:
            current_bbox = bbox.xyxyn[0]
            rel_x = current_bbox[0] + (current_bbox[2] - current_bbox[0]) / 2
            rel_y = current_bbox[1] + (current_bbox[3] - current_bbox[1]) / 2
            width = (
                current_bbox[2] - current_bbox[0] + random.randint(-0.1, 0.1)
            )
            height = (
                current_bbox[3] - current_bbox[1] + random.randint(-0.1, 0.1)
            )
            writer.writerow(
                [
                    file_name,
                    int(bbox.cls[0]),
                    float(rel_x),
                    float(rel_y),
                    float(width),
                    float(height),
                ]
            )